In [1]:
import jieba
jieba.set_dictionary('ChineseDictionary/dict.txt.big')
import numpy as np
import pandas as pd
import operator
import matplotlib.pyplot as plt
%matplotlib inline
data = pd.read_excel("customer_record_data.xlsx")

/home/louiechiu/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)


In [2]:
data.head()

,類別,Unique ID,資料日期,客戶事件描述
0,Call Out,AA2765,20180102 141000,[ECP] 去電確認開戶文件 客戶說後來去買美元保單就不開戶了
1,Call Out,AA3910,20180102 144300,[ECP]不方說 IPO MAIL
2,Call Out,AA3167,20180102 154500,[ECP] IPO 忙 先MAIL
3,Call Out,AA2355,20180102 194359,家庭戶 媽媽說女兒不開了，因為打工收入不穩定
4,Call Out,AA2778,20180103 111132,推Q1


First, lets see how many unique IDs are there in the record

In [3]:
data = np.array(data)
unique_ids = set(data[:,1])
print("Amount of Unique IDs: "+str(len(unique_ids)))

Amount of Unique IDs: 572


Then we count times of the ID (we will call it customer) had called.

In [4]:
unique_id_called = dict()
for i in data:
    if i[1] in unique_id_called:
        unique_id_called[i[1]] += 1
    else:
        unique_id_called[i[1]] = 1

sorted_unique_id_called = sorted(unique_id_called.items(),key=operator.itemgetter(1))
# for i in range(1,len(unique_id_called)):
#     print(sorted_unique_id_called[-i])

draw_hist = dict()
for i in sorted_unique_id_called:
    if i[1] in draw_hist:
        draw_hist[i[1]] += 1
    else:
        draw_hist[i[1]] = 1
        
keys = np.array(list(draw_hist.keys()))
values = np.array(list(draw_hist.values( ) ) )
for i in range(len(keys)):
    print("called",keys[i],"time(s) for",values[i],"times")


called 1 time(s) for 350 times
called 2 time(s) for 123 times
called 3 time(s) for 50 times
called 4 time(s) for 20 times
called 5 time(s) for 15 times
called 6 time(s) for 8 times
called 7 time(s) for 4 times
called 8 time(s) for 1 times
called 14 time(s) for 1 times


Now, we look at some of the most common word in the message section.

In [5]:
import jieba.analyse
jieba.enable_parallel(8)
cnt = 0
for i in data:
    cnt = cnt + 1
    if cnt <= 15:
        print(jieba.lcut(i[3]))

Building prefix dict from /home/louiechiu/DataScience/HW1/ChineseDictionary/dict.txt.big ...
Dumping model to file cache /tmp/jieba.ufef726f4388956d883eb10c0b6214106.cache
Loading model cost 1.610 seconds.
Prefix dict has been built succesfully.


['[', 'ECP', ']', ' ', '去電', '確認', '開戶', '文件', ' ', '客戶', '說', '後來', '去', '買', '美元', '保單', '就', '不', '開戶', '了']
['[', 'ECP', ']', '不方', '說', ' ', ' ', 'IPO', ' ', 'MAIL']
['[', 'ECP', ']', ' ', 'IPO', ' ', '忙', ' ', '先', 'MAIL']
['家庭', '戶', ' ', '媽媽', '說', '女兒', '不開', '了', '，', '因為', '打工', '收入', '不', '穩定']
['推', 'Q1']
['兒子', '要', '開戶', '未成年', ',', '不願辦', '網路', '要', '單筆', '買', '亞太', '複合', '債', '.']
['林', 'O', '屏', '216022', ' ', '家庭', '戶', ' ', '有', '可能', '有', '這戶', '下單', '又', '嫌', '麻煩']
['台幣', ' ', '瑞興', ' ', '轉介', '208']
['想', '買', '基金', ' ', '推', 'EC', ' ', '轉介', '之']
['某', '小姐', '要', '買', 'nnl', '新興', '市場', '債', ' ', '美元', '買', ' ']
['[', 'ECP', ']', '可能', '買回', ' ', ' ', '積', ' ', '單', ' ', '可以', '忍', '波動']
['Idris', '轉介']
['客戶', '來電', '說', '看到', '4', '年', 'IPO', ' ', '要', '美金', '的', ' ', '先寄', '開戶', '文件']
['去電', '確認', '外幣', '帳號', '是', '台北', '富邦']
['要', '用', '女兒', '名字', '開']


From the above we can see what is segmented. It is apparently some kind of junk. SO, instead, we search for keywords.

In [6]:
key_words = dict()
cnt = 0
for i in data:
    for j in jieba.analyse.extract_tags(i[3]):
        if j in key_words:
            key_words[j] += 1
        else:
            key_words[j] = 1
    cnt = cnt + 1
    if cnt <=15:
        print(jieba.analyse.extract_tags(i[3]))

key_words = sorted(key_words.items(),key=operator.itemgetter(1))

['開戶', 'ECP', '去電', '確認', '客戶', '後來', '保單', '文件', '美元']
['ECP', '不方', 'IPO', 'MAIL']
['ECP', 'IPO', 'MAIL']
['媽媽', '女兒', '不開', '因為', '穩定', '打工', '家庭', '收入']
['Q1']
['兒子', '開戶', '不願辦', '網路', '單筆', '亞太', '複合', '未成年']
['216022', '這戶', '下單', '麻煩', '家庭', '可能']
['台幣', '瑞興', '轉介', '208']
['EC', '轉介', '基金']
['nnl', '新興', '市場', '小姐', '美元']
['ECP', '買回', '波動', '可能', '可以']
['Idris', '轉介']
['先寄', '客戶', '來電', 'IPO', '開戶', '美金', '文件', '看到']
['去電', '確認', '外幣', '帳號', '富邦', '台北']
['女兒', '名字']


Now the above seems much more readable, although we may lose some of the information in the cutting stage. Some first thoughts upon seeing these datas.
1. It is mostly key word oriented and thus many verbs are removed
2. Given that it does mention some commodities, it is still hard to decide its intention.
    * For example, '覺得', '計價', '6%', '收益', '美金', '短期', '全球' we don't know whether the customer wants to long or short 美金.
3. There are unimportant verbs concerning selling commodities.
    * 解釋, '收到', '請客'
4. Abbreviations:
    * 國世銀 $\rightarrow$ 國泰世華銀行 ，富邦$\rightarrow$ 富邦銀行
    
The following blocks will show the most commonly appeared word in the service record.

In [7]:
for i in range(1,20):
    
    print(key_words[-i])

('ECP', 441)
('名單', 180)
('開戶', 163)
('Money', 148)
('客戶', 122)
('專刊', 92)
('退休', 84)
('訂閱', 78)
('電子報', 78)
('寄送', 76)
('追蹤', 74)
('分配', 65)
('IPO', 65)
('EC', 62)
('轉介', 59)
('寄回', 57)
('家庭', 54)
('收到', 47)
('表單', 46)


The above method doesn't seems to be a good idea. Since, applying key word count to different customer is useless.
Lets look at what each customer is most interested in. Here we set a limit that a **regular** customer are those who at least called $3$ times.

In [29]:
regular_customer = []
regular_thresh = 0
for i in unique_id_called:
    if unique_id_called[i] >= regular_thresh:
#         print("hit")
        regular_customer.append(i)
regular_customer = np.array(regular_customer)
regular_customer

array(['AA2765', 'AA3910', 'AA3167', 'AA2355', 'AA2778', 'AA2825',
       'AA2826', 'AA2827', 'AA2828', 'AA4226', 'AA2829', 'AA2784',
       'AA2833', 'AA5785', 'AA7870', 'AA8654', 'AA2834', 'AA2749',
       'AA2841', 'AA4209', 'AA2843', 'AA8011', 'AA5706', 'AA2844',
       'AA2771', 'AA2453', 'AA2758', 'AA2847', 'AA2849', 'AA2850',
       'AA2852', 'AA0809', 'AA0263', 'AA2854', 'AA2855', 'AA2856',
       'AA2831', 'AA2857', 'AA2858', 'AA2553', 'AA2861', 'AA2863',
       'AA2866', 'AA2867', 'AA2871', 'AA9022', 'AA6578', 'AA2872',
       'AA1160', 'AA2873', 'AA2851', 'AA3548', 'AA2874', 'AA2846',
       'AA2876', 'AA2879', 'AA2285', 'AA2286', 'AA2881', 'AA2883',
       'AA2884', 'AA2886', 'AA8199', 'AA2888', 'AA4291', 'AA2890',
       'AA4372', 'AA2892', 'AA2893', 'AA2885', 'AA2895', 'AA2896',
       'AA2868', 'AA2899', 'AA2900', 'AA2901', 'AA2902', 'AA2904',
       'AA2903', 'AA6370', 'AA2869', 'AA2905', 'AA2906', 'AA2910',
       'AA2913', 'AA2912', 'AA2917', 'AA2920', 'AA1377', 'AA48

In [58]:
df1 = pd.DataFrame(list(unique_id_called.keys()),columns=['Customer ID'])
df2 = pd.DataFrame(list(unique_id_called.values()),columns=['Call time'])
df3 = pd.DataFrame(list("abc"),columns=['tags'])
merged_df = pd.concat([df1,df2,df3],axis=1)

merged_df.head()


,Customer ID,Call time,tags
0,AA2765,1,a
1,AA3910,3,b
2,AA3167,3,c
3,AA2355,3,NaN
4,AA2778,1,NaN


In [62]:
cnt = 0
print_limt = 15 # update this to print more customers

for i in regular_customer:
    print(i)
    
    tag = list()
    # go through the data and find all those that matches "i"
    for j in data:
        if j[1] == i:
            tag.append(jieba.analyse.extract_tags( j[3]))
    
    
    tag = [item for sublist in tag for item in sublist]
    print(merged_df.at[cnt,2])
    merged_df.at[cnt,2] = tag
    cnt += 1
    print(tag)


AA2765


ValueError: At based indexing on an non-integer index can only have non-integer indexers

From this step, lets make some **beautiful** dataframes.

# TODO
## prepare data into a tidy data frame, where colmnames are 1. ID 2. Call times 3. keywords of talk 4. list out the top-5 high keywords
## 

In [12]:
myarr = list(list())
print(myarr)


[]


IndexError: list index out of range